In [1]:
import numpy as np
from numpy import array
import pandas as pd
from pandas import DataFrame

In [2]:
rain_raw = pd.read_csv('/home/lzy/lab_env/Project/太湖流域数据/原始数据/雨量-湖西区5.csv')
print(rain_raw.shape)
rain_raw.columns

(953948, 6)


Index(['STCD', 'TM', 'DRP', 'INTV', 'PDR', 'DYP'], dtype='object')

### 检查空值

In [3]:
rain_raw.isnull().any()


STCD    False
TM      False
DRP      True
INTV     True
PDR      True
DYP      True
dtype: bool

### 获取雨量站数据

In [4]:
# rainfall = rainfall.groupby('STCD')
rain_sts = rain_raw.groupby('STCD')

In [5]:
id_list = []
other_list = []
for i,j in rain_sts:
    # print(i,j)
    id_list.append(i)
    other_list.append(j.iloc[:,[0,1,2,5]])
print(id_list)
print(other_list[1])

[63125200, 63125500, 63125700, 63125800, 63125830, 63202501, 63202580, 63202600, 63202610, 63202650, 63203200, 63203351, 63224600]
            STCD                   TM  DRP  DYP
3751    63125500   2013-7-30 15:00:00  0.0  NaN
3755    63125500   2013-7-30 11:00:00  0.0  NaN
3758    63125500   2013-7-30 18:00:00  0.0  NaN
3761    63125500   2013-7-30 22:00:00  0.0  NaN
3765    63125500   2013-7-30 23:00:00  0.0  NaN
...          ...                  ...  ...  ...
953886  63125500  2022-11-16 16:00:00  0.0  NaN
953909  63125500   2022-11-16 2:00:00  0.0  NaN
953919  63125500   2022-11-16 8:00:00  0.0  0.0
953926  63125500  2022-11-16 13:00:00  0.0  NaN
953930  63125500   2022-11-16 3:00:00  0.0  NaN

[80965 rows x 4 columns]


In [6]:
other_list[1]['TM'] = pd.to_datetime(other_list[1]['TM'])
other_list[1] = other_list[1].sort_values(by='TM')
print(other_list[1])
print(other_list[1]['TM'].iloc[0])

            STCD                  TM  DRP  DYP
3897    63125500 2013-07-29 16:00:00  0.0  NaN
3850    63125500 2013-07-29 17:00:00  0.0  NaN
3853    63125500 2013-07-29 18:00:00  0.0  NaN
3954    63125500 2013-07-29 19:00:00  0.0  NaN
3966    63125500 2013-07-29 20:00:00  0.0  NaN
...          ...                 ...  ...  ...
953774  63125500 2022-11-16 12:00:00  0.0  NaN
953926  63125500 2022-11-16 13:00:00  0.0  NaN
953872  63125500 2022-11-16 14:00:00  0.0  NaN
953835  63125500 2022-11-16 15:00:00  0.0  NaN
953886  63125500 2022-11-16 16:00:00  0.0  NaN

[80965 rows x 4 columns]
2013-07-29 16:00:00


In [7]:
# start = other_list[1]['TM'].iloc[0]
# end = other_list[1]['TM'].iloc[-1]
# t_index = pd.date_range(start=start, end=end, freq='H')
# full_time = pd.DataFrame(t_index, columns=['TM'])
# full_time['STCD'] = id_list[1]
# print(len(full_time))
# complete = other_list[1].merge(full_time,how='right',sort=True)
# complete = complete.fillna(0)
# complete

### 时频补全

In [8]:
complete_list = []
for i in range(len(id_list)):
    #按照日期排序
    other_list[i]['TM'] = pd.to_datetime(other_list[i]['TM'])
    other_list[i] = other_list[i].sort_values(by='TM')
    #生成完整时间序列的dataframe,DRP值填0
    start = other_list[i]['TM'].iloc[0]
    end = other_list[i]['TM'].iloc[-1]
    t_index = pd.date_range(start=start, end=end, freq='H')
    full_time = pd.DataFrame(t_index, columns=['TM'])
    full_time['STCD'] = id_list[i]
    #补充原始具有缺失时刻的dataframe表
    # complete = full_time.append(other_list[i])# pd.concat([full_time, other_list[i]]) #
    # complete = complete.drop_duplicates(subset = ['TM']).sort_values(by=['TM']).reset_index(drop=True)
    complete = other_list[i].merge(full_time,how='right',sort=True)
    complete = complete.fillna(0)   

    complete_list.append(complete)
print(start)
print(complete_list[1])

2013-07-29 16:00:00
           STCD                  TM  DRP  DYP
0      63125500 2013-07-29 16:00:00  0.0  0.0
1      63125500 2013-07-29 17:00:00  0.0  0.0
2      63125500 2013-07-29 18:00:00  0.0  0.0
3      63125500 2013-07-29 19:00:00  0.0  0.0
4      63125500 2013-07-29 20:00:00  0.0  0.0
...         ...                 ...  ...  ...
81524  63125500 2022-11-16 12:00:00  0.0  0.0
81525  63125500 2022-11-16 13:00:00  0.0  0.0
81526  63125500 2022-11-16 14:00:00  0.0  0.0
81527  63125500 2022-11-16 15:00:00  0.0  0.0
81528  63125500 2022-11-16 16:00:00  0.0  0.0

[81529 rows x 4 columns]


/tmp/ipykernel_34324/25612196.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_list[i]['TM'] = pd.to_datetime(other_list[i]['TM'])


In [9]:
print(complete_list[0])
print(complete_list[12])

           STCD                  TM   DRP  DYP
0      63125200 2013-07-05 18:00:00   2.0  0.0
1      63125200 2013-07-05 19:00:00  14.5  0.0
2      63125200 2013-07-05 20:00:00   6.0  0.0
3      63125200 2013-07-05 21:00:00   2.0  0.0
4      63125200 2013-07-05 22:00:00   0.5  0.0
...         ...                 ...   ...  ...
82098  63125200 2022-11-16 12:00:00   0.0  0.0
82099  63125200 2022-11-16 13:00:00   0.0  0.0
82100  63125200 2022-11-16 14:00:00   0.0  0.0
82101  63125200 2022-11-16 15:00:00   0.0  0.0
82102  63125200 2022-11-16 16:00:00   0.0  0.0

[82103 rows x 4 columns]
           STCD                  TM  DRP  DYP
0      63224600 2013-07-29 16:00:00  0.0  0.0
1      63224600 2013-07-29 17:00:00  0.0  0.0
2      63224600 2013-07-29 18:00:00  0.0  0.0
3      63224600 2013-07-29 19:00:00  0.0  0.0
4      63224600 2013-07-29 20:00:00  0.0  0.0
...         ...                 ...  ...  ...
81524  63224600 2022-11-16 12:00:00  0.0  0.0
81525  63224600 2022-11-16 13:00:00  0.0  

In [11]:
stations = pd.concat(complete_list,axis=1)
stations

,STCD,TM,DRP,DYP,STCD,TM,DRP,DYP,STCD,TM,...,DRP,DYP,STCD,TM,DRP,DYP,STCD,TM,DRP,DYP
0,63125200.0,2013-07-05 18:00:00,2.0,0.0,63125500.0,2013-07-29 16:00:00,0.0,0.0,63125700.0,2013-07-29 16:00:00,...,1.0,0.0,63203351,2012-01-01 08:00:00,0.0,0.0,63224600.0,2013-07-29 16:00:00,0.0,0.0
1,63125200.0,2013-07-05 19:00:00,14.5,0.0,63125500.0,2013-07-29 17:00:00,0.0,0.0,63125700.0,2013-07-29 17:00:00,...,4.5,0.0,63203351,2012-01-01 09:00:00,0.0,0.0,63224600.0,2013-07-29 17:00:00,0.0,0.0
2,63125200.0,2013-07-05 20:00:00,6.0,0.0,63125500.0,2013-07-29 18:00:00,0.0,0.0,63125700.0,2013-07-29 18:00:00,...,1.0,0.0,63203351,2012-01-01 10:00:00,0.0,0.0,63224600.0,2013-07-29 18:00:00,0.0,0.0
3,63125200.0,2013-07-05 21:00:00,2.0,0.0,63125500.0,2013-07-29 19:00:00,0.0,0.0,63125700.0,2013-07-29 19:00:00,...,0.0,0.0,63203351,2012-01-01 11:00:00,0.0,0.0,63224600.0,2013-07-29 19:00:00,0.0,0.0
4,63125200.0,2013-07-05 22:00:00,0.5,0.0,63125500.0,2013-07-29 20:00:00,0.0,0.0,63125700.0,2013-07-29 20:00:00,...,0.0,0.0,63203351,2012-01-01 12:00:00,0.0,0.0,63224600.0,2013-07-29 20:00:00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95332,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaN,63203351,2022-11-16 12:00:00,0.0,0.0,NaN,NaT,NaN,NaN
95333,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaN,63203351,2022-11-16 13:00:00,0.0,0.0,NaN,NaT,NaN,NaN
95334,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaN,63203351,2022-11-16 14:00:00,0.0,0.0,NaN,NaT,NaN,NaN
95335,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaN,63203351,2022-11-16 15:00:00,0.0,0.0,NaN,NaT,NaN,NaN


In [12]:
stations.to_csv('/home/lzy/lab_env/Project/太湖流域数据/处理数据/时频/雨量-湖西区5-整编后.csv')